## 영화 5개 받기

In [1]:
print("좋아하는 영화와 그 개봉연도를 ,로 구분하여 5개 입력해주세요(ex. 명량 2014, 어벤져스 2018): ")
movie_list = input().split(",")
print(movie_list)

좋아하는 영화와 그 개봉연도를 ,로 구분하여 5개 입력해주세요(ex. 명량 2014, 어벤져스 2018): 
모가디슈 2021, 천문 2019, 라라랜드 2016, 크루엘라 2021, 블랙 위도우 2021
['모가디슈 2021', ' 천문 2019', ' 라라랜드 2016', ' 크루엘라 2021', ' 블랙 위도우 2021']


## 영화 정보

In [2]:
import requests
import json

In [3]:
movie_cd_list = []
director_list = []
selected_movie_list = []

for movie in movie_list:
    movie_cd_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key='+키+'&movieNm='+movie[:-5]
    res = requests.get(movie_cd_url)
    text = res.text
    d = json.loads(text)
    if d['movieListResult']['totCnt'] > 1:    # 동명의 영화가 있을 경우
        item_year = movie[-4:]   # 개봉연도로 구분
        for item in d['movieListResult']['movieList']:
            if item_year == item['openDt'][:4]:
                # director_list = director['peopleNm']
                movie_cd_list.append([item['movieCd'],
                                      item['movieNm'],
                                      item['genreAlt'],
                                      item['openDt'][:4]])
                selected_movie_list.append(item['movieNm'])
    if d['movieListResult']['totCnt'] == 1:
        for item in d['movieListResult']['movieList']:
            movie_cd_list.append([item['movieCd'],
                                  item['movieNm'],
                                  item['genreAlt'],
                                  item['openDt'][:4]])
            selected_movie_list.append(item['movieNm'])

print(movie_cd_list)
print(selected_movie_list)

[['20204117', '모가디슈', '액션,드라마', '2021'], ['20184571', '천문: 하늘에 묻는다', '사극', '2019'], ['20167904', '라라랜드', '드라마,뮤지컬,멜로/로맨스', '2016'], ['20216362', '크루엘라', '드라마,범죄,코미디', '2021'], ['20191951', '블랙 위도우', '액션,어드벤처,SF', '2021']]
['모가디슈', '천문: 하늘에 묻는다', '라라랜드', '크루엘라', '블랙 위도우']


In [4]:
import pandas as pd

movie_cdlist_df = pd.DataFrame(movie_cd_list)
movie_cdlist_df.columns = ['movieCd', 'movieNm','genreAlt', 'openYr']
movie_cdlist_df

,movieCd,movieNm,genreAlt,openYr
0,20204117,모가디슈,"액션,드라마",2021
1,20184571,천문: 하늘에 묻는다,사극,2019
2,20167904,라라랜드,"드라마,뮤지컬,멜로/로맨스",2016
3,20216362,크루엘라,"드라마,범죄,코미디",2021
4,20191951,블랙 위도우,"액션,어드벤처,SF",2021


## 영화코드로 정보 추출

In [5]:
movie_detail_list = []
movie_director_list = []
movie_cast_list = []    # 한 영화당 최대 10명의 배우를 추출하여 저장한 리스트
direct_list = []
movie_mainactor_list = []    # 한 영화당 최대 2명(주인공)의 배우를 추출하여 저장한 리스트
movie_genre_list = [] 

actor_list = []              # 5개 영화의 movie_cast_list을 모은 것
mainactor_list = []          # 5개 영화의 movie_mainactor_list을 모은 것
genre_list = []              # 5개 영화의 장르를 모은 것

movie_actor_list = [[] for i in range(0,5)]
movie_direc_list = [[] for i in range(0,5)]
j=0

for movieCd in movie_cdlist_df['movieCd']:
    movie_director_list.clear()
    movie_cast_list.clear()
    movie_mainactor_list.clear()
    movie_genre_list.clear()
    actor_list_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='+키+'&movieCd='+movieCd
    res = requests.get(actor_list_url)
    text = res.text
    d = json.loads(text)

    for direc in d['movieInfoResult']['movieInfo']['directors']:
        movie_director_list.append(direc['peopleNm'])
        direct_list+=movie_director_list

    for cast in d['movieInfoResult']['movieInfo']['actors']:
        if len(movie_mainactor_list) < 3:
            movie_mainactor_list.append(cast['peopleNm'])
        if len(movie_cast_list) <= 9:
            movie_cast_list.append(cast['peopleNm'])
            
    for genr in d['movieInfoResult']['movieInfo']['genres']:
        movie_genre_list.append(genr['genreNm'])
  
    #print(movie_cast_list)
    actor_list += movie_cast_list
    mainactor_list += movie_mainactor_list
    genre_list += movie_genre_list

    j+=1

print(direct_list)
print(actor_list)
print(mainactor_list)
print(genre_list)

['류승완', '허진호', '데이미언 셔젤', '크레이그 질레스피', '케이트 쇼트랜드']
['김윤석', '조인성', '허준호', '구교환', '김소진', '정만식', '김재화', '박경혜', '박명신', '한철우', '최민식', '한석규', '신구', '김홍파', '허준호', '김태우', '김원해', '임원희', '오광록', '박성훈', '엠마 스톤', '라이언 고슬링', '핀 위트록', 'J.K. 시몬스', '로즈마리 드윗', '엠마 스톤', '엠마 톰슨', '마크 스트롱', '스칼렛 요한슨', '플로렌스 퓨', '레이첼 와이즈', '데이빗 하버']
['김윤석', '조인성', '허준호', '최민식', '한석규', '신구', '엠마 스톤', '라이언 고슬링', '핀 위트록', '엠마 스톤', '엠마 톰슨', '마크 스트롱', '스칼렛 요한슨', '플로렌스 퓨', '레이첼 와이즈']
['액션', '드라마', '사극', '드라마', '뮤지컬', '멜로/로맨스', '드라마', '범죄', '코미디', '액션', '어드벤처', 'SF']


## 배우 등장 횟수

In [6]:
actor_dict = dict()
for i in actor_list:
    if i not in actor_dict:
        actor_dict[i]=1
    else:
        actor_dict[i]=actor_dict[i]+1

actor_dict

{'김윤석': 1,
 '조인성': 1,
 '허준호': 2,
 '구교환': 1,
 '김소진': 1,
 '정만식': 1,
 '김재화': 1,
 '박경혜': 1,
 '박명신': 1,
 '한철우': 1,
 '최민식': 1,
 '한석규': 1,
 '신구': 1,
 '김홍파': 1,
 '김태우': 1,
 '김원해': 1,
 '임원희': 1,
 '오광록': 1,
 '박성훈': 1,
 '엠마 스톤': 2,
 '라이언 고슬링': 1,
 '핀 위트록': 1,
 'J.K. 시몬스': 1,
 '로즈마리 드윗': 1,
 '엠마 톰슨': 1,
 '마크 스트롱': 1,
 '스칼렛 요한슨': 1,
 '플로렌스 퓨': 1,
 '레이첼 와이즈': 1,
 '데이빗 하버': 1}

## 주연배우(맨 앞 3사람)

In [7]:
mainactor_dict = dict()
for i in mainactor_list:
    if i not in mainactor_dict:
        mainactor_dict[i]=1
    else:
        mainactor_dict[i]=mainactor_dict[i]+1

mainactor_dict

{'김윤석': 1,
 '조인성': 1,
 '허준호': 1,
 '최민식': 1,
 '한석규': 1,
 '신구': 1,
 '엠마 스톤': 2,
 '라이언 고슬링': 1,
 '핀 위트록': 1,
 '엠마 톰슨': 1,
 '마크 스트롱': 1,
 '스칼렛 요한슨': 1,
 '플로렌스 퓨': 1,
 '레이첼 와이즈': 1}

## 추천배우 5명

- 2번 등장 배우 우선
- 빈자리는 랜덤

In [8]:
import random

prefer_actor_list = []
for key, value in actor_dict.items():
    if value > 1:
        prefer_actor_list.append(key)
if len(prefer_actor_list)!=5:
    while True:
        actor = random.choice(mainactor_list)
        if actor not in prefer_actor_list:
            prefer_actor_list.append(actor)
        if len(prefer_actor_list)==5:
            break

prefer_actor_list

['허준호', '엠마 스톤', '김윤석', '라이언 고슬링', '마크 스트롱']

## 배우 추천 코드

In [9]:
actor_code = []
filmo_list = []
final_filmo_list = []

for prefer_actor in prefer_actor_list:
    actor_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key='+키+'&peopleNm='+prefer_actor
    res = requests.get(actor_list_url)
    text = res.text
    d = json.loads(text)

    for actor in d['peopleListResult']['peopleList']:
        final_filmo_list.clear()
        if actor['repRoleNm'] == '배우' and actor['peopleNmEn'] != "":
            filmo_list = actor['filmoNames'].split("|")
            for filmo in filmo_list:
                if filmo not in selected_movie_list:
                    final_filmo_list.append(filmo)
            actor_code.append([actor['peopleCd'], actor['peopleNm'],actor['peopleNmEn'], actor['repRoleNm'], final_filmo_list[:5]])

In [10]:
print(selected_movie_list)
print(filmo_list)
print(final_filmo_list)

['모가디슈', '천문: 하늘에 묻는다', '라라랜드', '크루엘라', '블랙 위도우']
['크루엘라', '1917', '샤잠!', '더 캐쳐 워즈 어 스파이', '킹스맨: 골든 서클', '대테러 님로드 작전', '미스 슬로운', '내가 잠들기 전에', '마인드스케이프', '블러드 워', '테이크다운', '팅커 테일러 솔저 스파이', '킥 애스: 영웅의 탄생', '바디 오브 라이즈', '투 엔드 올 워즈', '슈퍼스티션', '이프 온리: 맨 위드 레인']
['1917', '샤잠!', '더 캐쳐 워즈 어 스파이', '킹스맨: 골든 서클', '대테러 님로드 작전', '미스 슬로운', '내가 잠들기 전에', '마인드스케이프', '블러드 워', '테이크다운', '팅커 테일러 솔저 스파이', '킥 애스: 영웅의 탄생', '바디 오브 라이즈', '투 엔드 올 워즈', '슈퍼스티션', '이프 온리: 맨 위드 레인']


In [11]:
actor_code_df = pd.DataFrame(actor_code)
actor_code_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm', 'filmoNames']
actor_code_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,10088453,허준호,HUH Joon-ho,배우,"[노량: 죽음의 바다, 결백, 퍼펙트맨, 국가부도의 날, 인랑]"
1,10049633,엠마 스톤,Emma Stone,배우,"[크루즈 패밀리: 뉴 에이지, 좀비랜드: 더블 탭, 더 페이버릿: 여왕의 여자, 빌..."
2,10005276,김윤석,KIM Yun-seok,배우,"[노량: 죽음의 바다, 바이러스(가제), 미성년, 암수살인, 1987]"
3,10014012,라이언 고슬링,Ryan Gosling,배우,"[퍼스트맨, 블레이드 러너 2049, 송 투 송, 나이스 가이즈, 빅쇼트]"
4,10024254,마크 스트롱,Mark Strong,배우,"[1917, 샤잠!, 더 캐쳐 워즈 어 스파이, 킹스맨: 골든 서클, 대테러 님로드..."


## 장르 등장 횟수 

In [12]:
genre_dict = dict()
for i in genre_list:
    if i not in genre_dict:
        genre_dict[i]=1
    else:
        genre_dict[i]=genre_dict[i]+1

genre_dict

{'액션': 2,
 '드라마': 3,
 '사극': 1,
 '뮤지컬': 1,
 '멜로/로맨스': 1,
 '범죄': 1,
 '코미디': 1,
 '어드벤처': 1,
 'SF': 1}

In [13]:
genre_df = pd.DataFrame(genre_dict.items())
genre_df.columns=['genres', 'number']
genre_df

,genres,number
0,액션,2
1,드라마,3
2,사극,1
3,뮤지컬,1
4,멜로/로맨스,1
5,범죄,1
6,코미디,1
7,어드벤처,1
8,SF,1


In [14]:
gen=dict()
gen=genre_df['genres']
gg=gen.values
gg

array(['액션', '드라마', '사극', '뮤지컬', '멜로/로맨스', '범죄', '코미디', '어드벤처', 'SF'],
      dtype=object)

In [15]:
recom_genre=[]
recom_genre=genre_df[genre_df['number']>1]
recom_genre_list=recom_genre['genres']
recom_genre_list

0     액션
1    드라마
Name: genres, dtype: object

In [16]:
recom_genre_list=recom_genre_list.values
recom_genre_list

array(['액션', '드라마'], dtype=object)

## 추려진 영화 장르 검색

In [17]:
genre_search=[]
movieID=[]
prdY=[]
odt=[]
movieN=[]
moviepeople=[]

for i in range(len(final_filmo_list)):
    genre_search_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key='+키+'&movieNm='+final_filmo_list[i]
    res = requests.get(genre_search_url)
    text = res.text
    d = json.loads(text)

    if d['movieListResult']['totCnt'] > 1:
        for item in d['movieListResult']['movieList']:
            movieID.append(item['movieCd'])

    if d['movieListResult']['totCnt'] == 1:        
        for item in d['movieListResult']['movieList']:
            genre_search.append([item['movieNm'], item['genreAlt']])
            
for i in range(len(movieID)):
    id_search_url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='+키+'&movieCd='+movieID[i]
    res = requests.get(id_search_url)
    text = res.text
    d = json.loads(text)
    
    dd=d['movieInfoResult']['movieInfo']
    for item in d['movieInfoResult']['movieInfo']['actors']:
        moviepeople=item['peopleNm']
        if moviepeople in prefer_actor_list:
            movieN.append(dd['movieNm'])
            prdY.append(dd['prdtYear'])
            odt.append(dd['openDt'])

for i in range(len(movieN)):
    duplit_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key='+키+'&movieNm='+movieN[i]
    res = requests.get(duplit_url)
    text = res.text
    d = json.loads(text)
    
    for item in d['movieListResult']['movieList']:
        if item['openDt']==odt[i]:
            genre_search.append([item['movieNm'], item['genreAlt']])
            break
        if item['prdtYear']==prdY[i]:
            genre_search.append([item['movieNm'], item['genreAlt']])

print(genre_search)

[['더 캐쳐 워즈 어 스파이', '드라마,전쟁'], ['킹스맨: 골든 서클', '액션,어드벤처,코미디'], ['대테러 님로드 작전', '액션,드라마'], ['미스 슬로운', '드라마,스릴러'], ['내가 잠들기 전에', '미스터리,스릴러'], ['마인드스케이프', '스릴러'], ['팅커 테일러 솔저 스파이', '스릴러'], ['투 엔드 올 워즈', '드라마,전쟁,액션'], ['이프 온리: 맨 위드 레인', '코미디,드라마,멜로/로맨스'], ['1917', '드라마,전쟁'], ['샤잠!', '액션'], ['블러드 워', '액션,스릴러'], ['테이크다운', '액션'], ['킥 애스: 영웅의 탄생', '액션,어드벤처,코미디'], ['바디 오브 라이즈', '스릴러,액션'], ['슈퍼스티션', '공포(호러)']]


In [18]:
pre_recom_pd=pd.DataFrame(genre_search)
pre_recom_pd.columns=['movieNm', 'genres']
pre_recom_pd

,movieNm,genres
0,더 캐쳐 워즈 어 스파이,"드라마,전쟁"
1,킹스맨: 골든 서클,"액션,어드벤처,코미디"
2,대테러 님로드 작전,"액션,드라마"
3,미스 슬로운,"드라마,스릴러"
4,내가 잠들기 전에,"미스터리,스릴러"
5,마인드스케이프,스릴러
6,팅커 테일러 솔저 스파이,스릴러
7,투 엔드 올 워즈,"드라마,전쟁,액션"
8,이프 온리: 맨 위드 레인,"코미디,드라마,멜로/로맨스"
9,1917,"드라마,전쟁"


## 추천 장르로 영화 추출

In [19]:
recom_genre=dict()
real=dict()
for i in range(len(recom_genre_list)):
    recom_genre=pre_recom_pd[pre_recom_pd['genres'].str.contains(recom_genre_list[i])]['movieNm']
    real.update(recom_genre)
real

{1: '킹스맨: 골든 서클',
 2: '대테러 님로드 작전',
 7: '투 엔드 올 워즈',
 10: '샤잠!',
 11: '블러드 워',
 12: '테이크다운',
 13: '킥 애스: 영웅의 탄생',
 14: '바디 오브 라이즈',
 0: '더 캐쳐 워즈 어 스파이',
 3: '미스 슬로운',
 8: '이프 온리: 맨 위드 레인',
 9: '1917'}

In [22]:
recommendation=[]
for key, value in real.items():
    recommendation.append(value)
recommendation

['킹스맨: 골든 서클',
 '대테러 님로드 작전',
 '투 엔드 올 워즈',
 '샤잠!',
 '블러드 워',
 '테이크다운',
 '킥 애스: 영웅의 탄생',
 '바디 오브 라이즈',
 '더 캐쳐 워즈 어 스파이',
 '미스 슬로운',
 '이프 온리: 맨 위드 레인',
 '1917']

## 영화추천

In [23]:
if len(recommendation)<5:
    recommendation=random.sample(recommendation, len(recommendation))
else:
    recommendation=random.sample(recommendation, 5)
recommendation

['킹스맨: 골든 서클', '바디 오브 라이즈', '투 엔드 올 워즈', '대테러 님로드 작전', '이프 온리: 맨 위드 레인']